In [5]:
from google.colab import drive
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertModel
import torch.nn as nn
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm
import torch.nn as nn
from sklearn.metrics import classification_report
import joblib

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
drive.mount('/content/drive')
print(os.listdir('/content/drive/My Drive'))


['midterm 2 review physixs 1A.pdf', 'midterm 2 notes (1).pdf', 'midterm 2 notes (2).pdf', 'final note physics 1A.pdf', 'Final 151B notes.pdf', 'S23_final_equation_sheet 2.pdf', 'Xiaochao.pdf', 'STATS 199 Contract(Yijie Lu).gdoc', 'STATS 199 Contract(Yijie Lu) (1).docx', 'Questions from Professor Zes.gdoc', '推荐信信息.gdoc', 'Cornell ORIE interveiew.gdoc', 'Colab Notebooks', 'Untitled0.ipynb', 'archive (2)', 'chatbot.ipynb', 'Untitled document (1).gdoc', 'script.gdoc', 'cs245-project-crag', 'Untitled', 'crag_task_1_dev_v4_release.jsonl.bz2', 'Untitled document.gdoc']


In [ ]:


# Read the file in chunks
chunk_size = 1000
file_path = '/content/drive/My Drive/crag_task_1_dev_v4_release.jsonl.bz2'

chunks = pd.read_json(file_path, compression='bz2', lines=True, chunksize=chunk_size)

# Initialize an empty list to store processed chunks
data_list = []

# Process each chunk
for i, chunk in enumerate(chunks):
    # Append to the list
    data_list.append(chunk[['query', 'domain']].dropna())

# Combine all chunks into a single DataFrame
data = pd.concat(data_list, ignore_index=True)
print(f"Total data size: {data.shape}")



Streaming output truncated to the last 5000 lines.
<ipython-input-3-d22ea3edfb9a>:13: FutureWarning: Parsed string "03/05/2024, 23:22:29 PT" included an un-recognized timezone "PT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  for i, chunk in enumerate(chunks):
<ipython-input-3-d22ea3edfb9a>:13: FutureWarning: Parsed string "03/12/2024, 12:29:17 PT" included an un-recognized timezone "PT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  for i, chunk in enumerate(chunks):
<ipython-input-3-d22ea3edfb9a>:13: FutureWarning: Parsed string "03/10/2024, 22:56:06 PT" included an un-recognized timezone "PT". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the str

Total data size: (2706, 2)


In [ ]:


# Encode the categorical target labels
label_encoders = {
    'domain': LabelEncoder()
}

for col in ['domain']:
    data[col] = label_encoders[col].fit_transform(data[col])

# Split into features (X) and targets (y)
X = data['query']
y = data[['domain']]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

print(f"Training examples: {len(X_train)}")
print(f"Testing examples: {len(X_test)}")



Training examples: 2435
Testing examples: 271


In [ ]:

class MultiTaskBERT(nn.Module):
    def __init__(self, num_domain_classes):
        super(MultiTaskBERT, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.domain_classifier = nn.Linear(self.bert.config.hidden_size, num_domain_classes)


    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output  # CLS token representation

        # Separate heads for each task
        domain_logits = self.domain_classifier(pooled_output)

        return domain_logits






In [ ]:

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:

class QueryDataset(Dataset):
    def __init__(self, queries, domain_labels):
        self.queries = queries
        self.domain_labels = domain_labels

    def __len__(self):
        return len(self.queries)

    def __getitem__(self, idx):
        # Tokenize the query
        inputs = tokenizer(
            self.queries[idx],
            truncation=True,
            padding='max_length',
            max_length=128,
            return_tensors='pt',
        )

        # Add labels to the output
        inputs = {key: val.squeeze(0) for key, val in inputs.items()}  # Remove extra dimension
        inputs['domain_labels'] = torch.tensor(self.domain_labels[idx], dtype=torch.long)
        return inputs



In [ ]:

# Convert labels and queries to lists
train_dataset = QueryDataset(
    queries=X_train.tolist(),
    domain_labels=y_train['domain'].tolist()
)

test_dataset = QueryDataset(
    queries=X_test.tolist(),
    domain_labels=y_test['domain'].tolist()
)

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [ ]:
for batch in train_dataloader:
    print(batch.keys())  # Should include 'input_ids', 'attention_mask', 'domain_labels', 'question_labels', 'dynamic_labels'
    print({key: val.shape for key, val in batch.items()})
    break


dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'domain_labels'])
{'input_ids': torch.Size([16, 128]), 'token_type_ids': torch.Size([16, 128]), 'attention_mask': torch.Size([16, 128]), 'domain_labels': torch.Size([16])}


In [ ]:


# Initialize model
num_domain_classes = len(y_train['domain'].unique())


model = MultiTaskBERT(num_domain_classes)

# Move model to device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Initialize optimizer and scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:

# Loss function
loss_fn = nn.CrossEntropyLoss()

# Training loop
num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(train_dataloader):
        # Move inputs and labels to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        domain_labels = batch['domain_labels'].to(device)

        # Forward pass
        domain_logits = model(input_ids, attention_mask)

        # Compute losses
        domain_loss = loss_fn(domain_logits, domain_labels)

        # Combine losses
        total_batch_loss = domain_loss
        total_loss += total_batch_loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        total_batch_loss.backward()
        optimizer.step()

    # Step the scheduler
    scheduler.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(train_dataloader)}")


100%|██████████| 153/153 [00:46<00:00,  3.27it/s]


Epoch 1/8, Loss: 0.5324758350167399


100%|██████████| 153/153 [00:48<00:00,  3.16it/s]


Epoch 2/8, Loss: 0.16928024030004452


100%|██████████| 153/153 [00:49<00:00,  3.10it/s]


...

Epoch 6/8, Loss: 0.04734747235593234


100%|██████████| 153/153 [00:49<00:00,  3.10it/s]


Epoch 7/8, Loss: 0.026146445379994224


100%|██████████| 153/153 [00:49<00:00,  3.10it/s]

Epoch 8/8, Loss: 0.031118420474276896


In [ ]:

model.eval()
all_domain_preds = []
all_domain_labels = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        # Move inputs and labels to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        domain_labels = batch['domain_labels'].to(device)

        # Get logits
        domain_logits = model(input_ids, attention_mask)

        # Get predictions
        domain_preds = torch.argmax(domain_logits, dim=1).cpu().numpy()

        # Append predictions and labels
        all_domain_preds.extend(domain_preds)

        all_domain_labels.extend(domain_labels.cpu().numpy())

# Classification reports
print("Domain Classification Report:")
print(classification_report(all_domain_labels, all_domain_preds))



100%|██████████| 17/17 [00:01<00:00,  8.82it/s]

Domain Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98        64
           1       0.95      0.97      0.96        63
           2       0.94      0.88      0.91        33
           3       0.88      0.88      0.88        51
           4       0.97      0.98      0.98        60

    accuracy                           0.95       271
   macro avg       0.94      0.94      0.94       271
weighted avg       0.95      0.95      0.95       271



In [ ]:
## save the model
torch.save(model.state_dict(), "multi_task_bert_model.pth")


In [ ]:
## save the encoder

for key, encoder in label_encoders.items():
    joblib.dump(encoder, f"{key}_label_encoder.pkl")


In [ ]:
## Load the Model and Encoders During Inference
model = MultiTaskBERT(num_domain_classes)
model.load_state_dict(torch.load("multi_task_bert_model.pth"))
model.eval()
model.to(device)


<ipython-input-15-8533425c5978>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("multi_task_bert_model.pth"))


MultiTaskBERT(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise

In [ ]:
## load the encoders
label_encoders = {
    'domain': joblib.load("domain_label_encoder.pkl")
}


In [ ]:

## Use the Model for Inference
def predict(query, model, tokenizer, label_encoders, device):
    # Tokenize the input query
    inputs = tokenizer(
        query,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt',
    )
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Forward pass through the model
    with torch.no_grad():
        domain_logits = model(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"]
        )

    # Get predictions
    domain_pred = torch.argmax(domain_logits, dim=1).item()
    # Decode predictions
    domain_label = label_encoders['domain'].inverse_transform([domain_pred])[0]

    return {
        "domain": domain_label
    }


In [ ]:
query = "Who is mozart?"
prediction = predict(query, model, tokenizer, label_encoders, device)
print(prediction)


{'domain': 'music'}


In [ ]:
print("Domain Classes:", label_encoders['domain'].classes_)

Domain Classes: ['finance' 'movie' 'music' 'open' 'sports']
